In [ ]:
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme="onedork", context="notebook", ticks=True, grid=True)

import numpy as np
import itertools
import h5py
import os
import sys
import glob

import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
import pandas as pd
import tikzplotlib

# from tqdm import tqdm
from tqdm.notebook import tqdm_notebook as tqdm

import helper.circular

In [ ]:
hist_bin = lambda n: np.linspace(0, np.pi, n + 1, endpoint=True)

df = pd.read_pickle(os.path.join("output/vs_0/voxel_size_simulation.pkl"))

for model in df.model.unique():

    fig, axs = plt.subplots(len(df.f0_inc.unique()),
                            len(df.voxel_size.unique()),
                            subplot_kw={'projection': 'polar'},
                            figsize=(20, 20))
    axs = np.atleast_2d(axs)

    for i, f0_inc in enumerate(sorted(df.f0_inc.unique())):
        for j, vs in enumerate(sorted(df.voxel_size.unique())):
            axs[i, j].set_xticklabels([])
            axs[i, j].set_yticklabels([])
            axs[i, j].set_title(f"f0:{f0_inc:.0f}, vs:{vs}")

    for i, f0_inc in enumerate(sorted(df.f0_inc.unique())):
        for j, vs in enumerate(sorted(df.voxel_size.unique())):
            sub = (df.omega == 0.0) & (df.psi == 1.0) & (df.f1_rot == 0.0) & (
                df.voxel_size == vs) & (df.f0_inc == f0_inc) & (df.model
                                                                == model)

            df_sub = df[sub]

            if len(df_sub) > 1:
                print("fooo", omega, psi)
            if len(df_sub) == 0:
                continue

            phi = df_sub.explode("f_phi").f_phi.to_numpy(float)
            theta = df_sub.explode("f_theta").f_theta.to_numpy(float)

            h, x = np.histogram(phi, hist_bin(4 * 18), density=True)
            x = x[:-1] + (x[1] - x[0]) / 2
            x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
            h = np.append(np.concatenate((h, h), axis=0), h[0])
            h = h / np.max(h)
            h = h[(x > np.pi * 3 / 2) | (x < np.pi / 2)]
            x = x[(x > np.pi * 3 / 2) | (x < np.pi / 2)]
            axs[i, j].plot(x, h)

            h, x = np.histogram(theta, hist_bin(10 * 18), density=True)
            x = np.pi - (np.pi / 2 - x[:-1] +
                         (x[1] - x[0]) / 2)  # np/2 for incl, - for plot
            h = h / np.max(h)
            h = h[x > np.pi / 2]
            x = x[x > np.pi / 2]
            axs[i, j].plot(x, h)

            phi = df_sub.explode("epa_dir").epa_dir.to_numpy(float)

            h, x = np.histogram(phi, hist_bin(2 * 18), density=True)
            x = x[:-1] + (x[1] - x[0]) / 2
            x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
            h = np.append(np.concatenate((h, h), axis=0), h[0])
            h = h / np.max(h)
            h = h[(x > np.pi * 3 / 2) | (x < np.pi / 2)]
            x = x[(x > np.pi * 3 / 2) | (x < np.pi / 2)]
            axs[i, j].plot(x, h, '--')

    #         h, x = np.histogram(theta, hist_bin(18), density=True)
    #         x = np.pi - (np.pi / 2 - x[:-1] +
    #                      (x[1] - x[0]) / 2)  # np/2 for incl, - for plot
    #         h = h / np.max(h)
    #         h = h[x > np.pi / 2]
    #         x = x[x > np.pi / 2]
    #         axs[i, j].plot(x, h)

In [ ]:
import fastpli.model.sandbox
import fastpli.model.solver
import fastpli.analysis

psi, omega, f0_inc, f1_rot = 1.0, 0, 45, 0
radius = 1

solver = fastpli.model.solver.Solver()
solver.obj_mean_length = radius * 2
solver.obj_min_radius = radius * 2
solver.omp_num_threads = 4

seeds = fastpli.model.sandbox.seeds.triangular_grid(60 * 2,
                                                    60 * 2,
                                                    1 * radius,
                                                    center=True)

# pick random seeds for fiber population distribution
seeds_0 = seeds[np.random.rand(seeds.shape[0]) < psi, :]
seeds_1 = seeds[np.random.rand(seeds.shape[0]) < (1 - psi), :]
rnd_radii_0 = radius * np.random.lognormal(0, 0.1, seeds_0.shape[0])
rnd_radii_1 = radius * np.random.lognormal(0, 0.1, seeds_1.shape[0])

vec = np.array([np.cos(np.deg2rad(omega)), np.sin(np.deg2rad(omega)), 0])
rot_inc = fastpli.tools.rotation.y(-np.deg2rad(f0_inc))
rot_phi = fastpli.tools.rotation.x(np.deg2rad(f1_rot))
rot = np.dot(rot_inc, rot_phi)
vec = np.dot(rot, vec)

fiber_bundles = [
    fastpli.model.sandbox.build.cuboid(p=-0.5 * np.array([60, 60, 60]),
                                       q=0.5 * np.array([60, 60, 60]),
                                       phi=np.deg2rad(0),
                                       theta=np.pi / 2 - np.deg2rad(f0_inc),
                                       seeds=seeds_0,
                                       radii=rnd_radii_0),
    fastpli.model.sandbox.build.cuboid(
        p=-0.5 * np.array([10, 10, 60 * 1.25]),
        q=0.5 * np.array([10, 10, 60 * 1.25]),
        #         phi=np.deg2rad(40),
        #         theta=np.deg2rad(35),
        phi=np.arctan2(vec[1], vec[0]),
        theta=np.arccos(vec[2]),
        seeds=seeds_1,
        radii=rnd_radii_1)
]

solver.fiber_bundles = fiber_bundles
fiber_bundles = solver.apply_boundary_conditions(100)
# add rnd displacement
for fb in fiber_bundles:
    for f in fb:
        f[:, :-1] += np.random.normal(0, 0.05 * radius, (f.shape[0], 3))
        f[:, -1] *= np.random.lognormal(0, 0.05, f.shape[0])

solver.fiber_bundles = fiber_bundles
solver.apply_boundary_conditions(100)
for i in tqdm(list(range(1, 50))):
    if solver.step():
        break

fiber_bundles = fastpli.objects.fiber_bundles.Cut(
    solver.fiber_bundles,
    [-0.5 * np.array([1, 1, 60]), 0.5 * np.array([1, 1, 60])])

fig, axs = plt.subplots(1, 1, subplot_kw={'projection': 'polar'})

phi, theta = fastpli.analysis.orientation.fiber_bundles(fiber_bundles)

h, x = np.histogram(phi, hist_bin(4 * 18), density=True)
x = x[:-1] + (x[1] - x[0]) / 2
x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
h = np.append(np.concatenate((h, h), axis=0), h[0])
h = h / np.max(h)
h = h[(x > np.pi * 3 / 2) | (x < np.pi / 2)]
x = x[(x > np.pi * 3 / 2) | (x < np.pi / 2)]
axs.plot(x, h)

h, x = np.histogram(theta, hist_bin(10 * 18), density=True)
x = np.pi - (np.pi / 2 - x[:-1] +
             (x[1] - x[0]) / 2)  # np/2 for incl, - for plot
h = h / np.max(h)
h = h[x > np.pi / 2]
x = x[x > np.pi / 2]
axs.plot(x, h)